In [2]:
# 🛠️ Install Gradio
!pip install gradio

# 📥 Import Libraries
import numpy as np
import gradio as gr
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 📚 Sample Training Text (you can replace this with your own)
text = """
alice was beginning to get very tired of sitting by her sister on the bank and of having nothing to do once or twice she had peeped into the book her sister was reading but it had no pictures or conversations in it and what is the use of a book thought alice without pictures or conversation
"""

# 🔠 Create character-to-index mappings
chars = sorted(list(set(text)))
char_to_idx = {c: i for i, c in enumerate(chars)}
idx_to_char = {i: c for c, i in char_to_idx.items()}

# 🔢 Prepare sequences
sequence_length = 40
step = 1
sequences = []
next_chars = []

for i in range(0, len(text) - sequence_length, step):
    sequences.append(text[i: i + sequence_length])
    next_chars.append(text[i + sequence_length])

# Encode sequences
X = np.zeros((len(sequences), sequence_length, len(chars)), dtype=np.bool_)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool_)

for i, seq in enumerate(sequences):
    for t, char in enumerate(seq):
        X[i, t, char_to_idx[char]] = 1
    y[i, char_to_idx[next_chars[i]]] = 1

# 🧠 Build LSTM Model
model = Sequential()
model.add(LSTM(128, input_shape=(sequence_length, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, batch_size=128, epochs=20, verbose=1)

# 🔮 Text generation function
def generate_text(seed, length=200, temperature=1.0):
    generated = seed.lower()
    seed = seed.lower().rjust(sequence_length)

    for _ in range(length):
        sampled = np.zeros((1, sequence_length, len(chars)))
        for t, char in enumerate(seed[-sequence_length:]):
            if char in char_to_idx:
                sampled[0, t, char_to_idx[char]] = 1.0

        preds = model.predict(sampled, verbose=0)[0]
        preds = np.asarray(preds).astype('float64')
        preds = np.log(preds + 1e-8) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        next_idx = np.random.choice(range(len(chars)), p=preds)
        next_char = idx_to_char[next_idx]

        generated += next_char
        seed += next_char

    return generated

# 🎨 Gradio UI
gr.Interface(
    fn=generate_text,
    inputs=[
        gr.Textbox(lines=1, placeholder="Enter a seed sentence...", label="Seed"),
        gr.Slider(50, 500, value=200, label="Generated Length"),
        gr.Slider(0.2, 1.5, value=1.0, label="Temperature (Creativity)")
    ],
    outputs="text",
    title="📝 Text Generator using LSTM",
    description="Enter a seed phrase and watch the LSTM generate the next part!"
).launch()


Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - loss: 3.1361
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 3.1138
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 3.0912
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 3.0595
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 3.0099
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 2.8971
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 2.8155
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 2.7709
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 2.7638
Epoch 10/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 2.7487
Epoch 11/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 2.7263
Epoch 12/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 2.7238
Epoch 13/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 2.7102
Epoch 14/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 2.7298
Epoch 15/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 2.7341
Epoch 16/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/